# Notebook to organize Ingestion of XQ100 datasets (v1.2)

In [1]:
# imports
import warnings
import pdb

from astropy import units as u
from astropy.table import Table
from astropy.coordinates import SkyCoord

from linetools.spectralline import AbsLine
from linetools import utils as ltu
from linetools.isgm.abscomponent import AbsComponent
from linetools.isgm import utils as ltiu
from linetools.analysis import absline as laa


import pyigm
from pyigm.surveys import dlasurvey as dlasvy
from pyigm.surveys import utils as pyiu
from pyigm.abssys.dla import DLASystem

## Meta Table

In [2]:
xqmeta_file = pyigm.__path__[0]+'/data/DLA/XQ-100/XQ100_v1_2.fits.gz'
xq100_meta = Table.read(xqmeta_file)

In [3]:
xq100_meta[0:5]

OBJ_NAME,RA,DEC,FILE_NAME,Z_QSO,ERR_ZQSO,N_OBS,MJD_OBS [3],SEEING_START [3],SEEING_END [3],SNR_1700 [3],SNR_3000 [3],SNR_3600 [3],CALIB_FLAG [3],COMBINED_FLAG,COMBINED_SNR_17,COMBINED_SNR_30,COMBINED_SNR_36,HR_FLAG,JOHNSON_MAG [3],SDSS_PSFMAG [5],SDSS_ERR_PSFMAG [5],DR7Q_MATCHED,PLATE_DR7Q,MJD_DR7Q,FIBER_DR7Q,DR12Q_MATCHED,N_DR12Q,PLATE_DR12Q [2],MJD_DR12Q [2],FIBER_DR12Q [2],FIRST_MATCHED,FIRST_FLUX,FIRST_SNR,MATCHED_TMASS,TMASS_MAG [3],TMASS_ERR_MAG [3],TMASS_SNR [3],RD_FLAG,MATCHED_WISE,WISE_MAG [4],WISE_ERR_MAG [4],WISE_SNR [4],WISE_RCHI2 [4],CC_FLAG,PH_QUAL,N_ABS,Z_ABS [4],NHI_ABS [4],ERR_NHI_ABS [4]
-,deg,deg,-,-,-,-,-,arcsec,arcsec,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,mJy,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,[cm-2],[cm-2]
str24,float64,float64,str1,float32,float32,int32,float32,float32,float32,float64,float64,float64,float64,int16,float64,float64,float64,int16,float32,float64,float64,int16,int32,int32,int32,int16,int32,int32,int32,int32,int32,float64,float64,int16,float64,float64,float64,str3,int16,float64,float64,float64,float64,str4,str3,int16,float64,float64,float64
HB89 0000-263,0.844985,-26.0554,-,4.12544,0.0002136,1,56219.1 .. -1.0,0.98 .. -1.0,1.45 .. -1.0,78.5073013306 .. -1.0,99.202003479 .. -1.0,-1.0 .. -1.0,0.0 .. -1.0,0,-1.0,-1.0,-1.0,1,18.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0,-1,-1,-1,0,0,-1 .. -1,-1 .. -1,-1 .. -1,-1,0.0,0.0,1,16.2700004578 .. 14.7779998779,0.11400000006 .. 0.107000000775,9.80000019073 .. 9.69999980927,222,1,13.9270000458 .. 7.47700023651,0.0280000008643 .. 0.134000003338,39.0 .. 8.10000038147,1.01900005341 .. 1.01800000668,0000,BCB,2,3.054500103 .. -1.0,20.2000007629 .. -1.0,0.10000000149 .. -1.0
BR J0006-6208,1.715033,-62.13355,-,4.43995,0.00089465,1,56209.2 .. -1.0,1.05 .. -1.0,1.43 .. -1.0,19.7355003357 .. -1.0,22.1355991364 .. -1.0,-1.0 .. -1.0,2.0 .. -1.0,0,-1.0,-1.0,-1.0,1,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0,-1,-1,-1,0,0,-1 .. -1,-1 .. -1,-1 .. -1,-1,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-,1,15.1999998093 .. 8.34200000763,0.0340000018477 .. 0.287999987602,32.2000007629 .. 3.79999995232,0.882799983025 .. 0.972999989986,0000,-,3,3.20300006866 .. -1.0,20.8999996185 .. -1.0,0.10000000149 .. -1.0
BR J0030-5159,7.643627,-51.49546,-,4.17286,0.00014623,1,56211.2 .. -1.0,0.82 .. -1.0,-1.0 .. -1.0,17.7509002686 .. -1.0,21.5370006561 .. -1.0,-1.0 .. -1.0,0.0 .. -1.0,0,-1.0,-1.0,-1.0,1,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0,-1,-1,-1,0,0,-1 .. -1,-1 .. -1,-1 .. -1,-1,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-,1,15.468000412 .. 8.77700042725,0.0370000004768 .. 0.0,29.7000007629 .. 0.0,0.965099990368 .. 0.0,,-,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0
PSS J0034+1639,8.727975,16.65506,-,4.29243,0.00019559,1,56219.1 .. -1.0,1.05 .. -1.0,1.34 .. -1.0,28.351600647 .. -1.0,30.108499527 .. -1.0,-1.0 .. -1.0,0.0 .. -1.0,0,-1.0,-1.0,-1.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0,-1,-1,-1,0,0,-1 .. -1,-1 .. -1,-1 .. -1,-1,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-,1,14.7530002594 .. 8.12699985504,0.0329999998212 .. 0.0,33.0 .. 0.0,0.85949999094 .. 0.0,,-,3,3.75500011444 .. -1.0,20.3999996185 .. -1.0,0.15000000596 .. -1.0
SDSS J004219.74-102009.4,10.582215,-10.33674,-,3.86293,0.00027549,1,56210.1 .. -1.0,0.58 .. -1.0,0.6 .. -1.0,51.7294006348 .. -1.0,48.0922012329 .. -1.0,57.8768005371 .. -1.0,0.0 .. -1.0,0,-1.0,-1.0,-1.0,1,20.57 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,0,-1,-1,-1,0,0,-1 .. -1,-1 .. -1,-1 .. -1,0,0.0,0.0,0,-1.0 .. -1.0,-1.0 .. -1.0,-1.0 .. -1.0,-,1,15.0579996109 .. 8.19699954987,0.0370000004768 .. 0.261000007391,29.2999992371 .. 4.19999980927,0.927299976349 .. 0.788500010967,0000,-,1,2.7555000782 .. -1.0,20.0 .. -1.0,0.10000000149 .. -1.0


In [4]:
xq100_coords = SkyCoord(ra=xq100_meta['RA'], dec=xq100_meta['DEC'], unit='deg')

## DLAs

### $N_{\rm HI}$ -- (Sanchez-Ramirez et al. 2016, MNRAS, 456, 4488)

In [5]:
xq100_dla = dlasvy.DLASurvey.load_XQ100(sample='all')
xq100_dla

Loading QSOs file /Users/xavier/local/Python/pyigm/pyigm/data/DLA/XQ-100/XQ100_zpath.fit


<IGMSurvey: nsys=41, type=DLA, ref=XQ-100, nsightlines=100>

In [6]:
xq100_dla._abs_sys[0:5]

[<DLASystem: 00:03:22.9104 -26:03:16.812, 3.39, NHI=21.4, Z/H=0>,
 <DLASystem: 00:06:51.6096 -62:08:03.696, 3.203, NHI=20.9, Z/H=0>,
 <DLASystem: 00:06:51.6096 -62:08:03.696, 3.775, NHI=21, Z/H=0>,
 <DLASystem: 00:34:54.8208 16:39:19.512, 3.755, NHI=20.4, Z/H=0>,
 <DLASystem: 01:13:44.3712 -28:03:17.208, 3.106, NHI=21.2, Z/H=0>]

In [7]:
xq100_dla_coords = xq100_dla.coord
xq100_dla_coords

<SkyCoord (ICRS): (ra, dec) in deg
    [(0.84546, -26.05467), (1.71504, -62.13436), (1.71504, -62.13436),
     (8.72842, 16.65542), (18.43488, -28.05478), (21.01571, 0.74244),
     (23.04104, 13.69431), (23.418, 4.01662), (23.418, 4.01662),
     (38.72975, -18.10236), (43.82741, 0.81318), (43.82741, 0.81318),
     (46.84533, -49.76333), (46.84533, -49.76333), (63.81321, -43.96469),
     (66.54304, -22.03814), (82.33671, -35.87614), (116.79647, 27.65093),
     (116.79647, 27.65093), (124.73242, 9.98001), (140.17401, 7.42892),
     (148.75039, -1.50181), (155.16926, 9.38174), (156.2359, 18.31911),
     (164.2724, 19.17857), (164.74326, 12.76526), (167.23113, 12.16482),
     (167.23113, 12.16482), (198.17863, 8.68475), (215.28232, -6.73232),
     (229.4841, 5.18433), (238.22935, 10.09399), (248.3318, 14.19501),
     (260.84667, 22.73278), (339.97333, -5.87194), (356.01333, 3.70722),
     (8.72842, 16.65542), (8.72842, 16.65542), (120.2095, 19.3497),
     (128.79551, 6.848), (238.22935, 10

In [8]:
xq100_dla_zabs = xq100_dla.zabs
xq100_dla_zabs

array([ 3.39  ,  3.203 ,  3.775 ,  3.755 ,  3.106 ,  2.261 ,  3.936 ,
        3.692 ,  3.7725,  3.693 ,  3.9145,  3.2555,  4.4665,  3.591 ,
        3.808 ,  2.9825,  3.684 ,  3.4235,  3.901 ,  3.306 ,  2.238 ,
        4.0245,  2.592 ,  2.298 ,  3.3735,  3.4315,  3.546 ,  3.3965,
        2.66  ,  3.449 ,  2.6885,  3.601 ,  2.882 ,  3.698 ,  4.0805,
        3.22  ,  4.2835,  4.2523,  3.9465,  3.9555,  3.6665])

#### Table 7

In [4]:
tab7_fil = pyigm.__path__[0]+'/data/DLA/XQ-100/sramirez16_table7.dat'
tab7 = Table.read(tab7_fil, format='ascii')
tab7[0:5]

z,Dz,sumDX,nabs,lx,bmlxlci_68.3,bmlxhci_68.3,bmlxlci_95.0,bmlxhci_95.0,Omega,bmOlci_68.3,bmOhci_68.3,bmOlci_95.0,bmOhci_95.0
float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2.15,0.831,965,62,0.064,0.056,0.072,0.049,0.081,0.543,0.506,0.753,0.432,0.983
2.16,0.831,997,64,0.064,0.056,0.072,0.049,0.081,0.53,0.494,0.729,0.422,0.948
2.17,0.83,1023,66,0.064,0.057,0.072,0.05,0.081,0.527,0.489,0.727,0.422,0.94
2.18,0.827,1094,68,0.062,0.055,0.07,0.048,0.078,0.532,0.495,0.721,0.425,0.917
2.19,0.826,1140,68,0.06,0.052,0.067,0.046,0.075,0.511,0.476,0.695,0.407,0.885


### Read Berg table

In [9]:
fil = pyigm.__path__[0]+'/data/DLA/XQ-100/tb_XQ100_metals_to_json.dat'
berg_tbl = Table.read(fil,format='ascii')
berg_tbl[0:5]

qso,ion,wl,zab,zem,ra,dec,vmin,vmax,filename,N,Nerr,flg,flg_analy,lambda,f
str10,str4,int64,float64,float64,str11,str11,int64,int64,str38,float64,float64,int64,int64,float64,float64
J0003-2603,OI,1302,3.39,4.125,00:03:22.79,-26:03:19.4,-60,70,J0003-2603/tb_J0003-2603_vis_cont.fits,15.1,0.02,3,-1,1302.168,0.04887
J0003-2603,MgI,2852,3.39,4.125,00:03:22.79,-26:03:19.4,-140,70,J0003-2603/tb_J0003-2603_nir_cont.fits,11.85,0.03,1,0,2852.964,1.81
J0003-2603,MgII,2796,3.39,4.125,00:03:22.79,-26:03:19.4,-220,200,J0003-2603/tb_J0003-2603_nir_cont.fits,13.91,0.02,2,0,2796.352,0.6123
J0003-2603,MgII,2803,3.39,4.125,00:03:22.79,-26:03:19.4,-200,170,J0003-2603/tb_J0003-2603_nir_cont.fits,14.04,0.01,2,0,2803.531,0.3054
J0003-2603,CaII,3934,3.39,4.125,00:03:22.79,-26:03:19.4,-20,50,J0003-2603/tb_J0003-2603_nir_cont.fits,11.81,0.0,3,0,3934.777,0.65


#### Systems

In [10]:
dlas = np.array(['{:s}_z{:0.3f}'.format(row['qso'],row['zab']) for row in berg_tbl])
unique_dlas = np.unique(dlas)
unique_dlas

array(['J0003-2603_z3.390', 'J0006-6208_z3.203', 'J0006-6208_z3.775',
       'J0034+1639_z3.755', 'J0034+1639_z4.252', 'J0034+1639_z4.284',
       'J0113-2803_z3.106', 'J0124+0044_z2.261', 'J0132+1341_z3.936',
       'J0134+0400_z3.692', 'J0134+0400_z3.772', 'J0234-1806_z3.693',
       'J0255+0048_z3.256', 'J0255+0048_z3.914', 'J0307-4945_z3.591',
       'J0307-4945_z4.466', 'J0415-4357_z3.808', 'J0424-2209_z2.982',
       'J0529-3552_z3.684', 'J0747+2739_z3.424', 'J0747+2739_z3.901',
       'J0800+1920_z3.946', 'J0818+0958_z3.306', 'J0835+0650_z3.955',
       'J0920+0725_z2.238', 'J0955-0130_z4.024', 'J1020+0922_z2.592',
       'J1024+1819_z2.298', 'J1057+1910_z3.373', 'J1058+1245_z3.432',
       'J1108+1209_z3.397', 'J1108+1209_z3.546', 'J1312+0841_z2.660',
       'J1421-0643_z3.449', 'J1517+0511_z2.688', 'J1552+1005_z3.601',
       'J1552+1005_z3.667', 'J1633+1411_z2.882', 'J1723+2243_z3.698',
       'J2239-0552_z4.080', 'J2344+0342_z3.220'], 
      dtype='|S17')

### Loop on DLAs and generate

#### TiII kludge

In [11]:
TiII = AbsLine('TiII 1910a')

/Users/xavier/local/Python/linetools/linetools/lists/linelist.py:374: RuntimeWarning: divide by zero encountered in log10
  self._data['log(w*f)'] = np.log10(qm_strength)
/Users/xavier/anaconda/lib/python2.7/site-packages/numpy/ma/core.py:824: RuntimeWarning: invalid value encountered in less_equal
  return umath.less_equal(x, self.critical_value)


Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [12]:
TiII.data['wrest'] = 1910.75*u.AA
TiII.data['f'] = 0.202
TiII.data['name'] = 'TiII 1910c'
TiII.name = 'TiII 1910c'
TiII.wrest = 1910.75*u.AA

In [13]:
tmp = TiII.copy()
TiIIa = AbsLine('TiII 1910a')

In [14]:
for cdla in unique_dlas:
    # Rows matching this DLA
    mti = np.where(dlas == cdla)[0]
    # Generate abslines
    abslines = []
    for jj,ii in enumerate(mti):
        row = berg_tbl[ii]
        if jj == 0:
            #if row['qso'] == 'J1517+0511':
            #    coord = ltu.radec_to_coord('J151756.2+051109.7')
            #else:
            coord = ltu.radec_to_coord((row['ra'],row['dec']))
            # Match to DLA in XQ-100 survey
            sep = coord.separation(xq100_dla_coords)
            imin = np.argmin(sep)
            mt = np.where((sep < 10*u.arcsec) & (np.abs(row['zab']-xq100_dla_zabs) < 1e-2))[0]
            if len(mt) != 1:
                pdb.set_trace()
            if sep[mt[0]] > 1*u.arcsec:
                warnings.warn("DLA {} has a large coordinate offset = {}".format(cdla,sep[mt].to('arcsec')))
            dla = xq100_dla._abs_sys[mt[0]]
            # Reset DLA
            dla.zabs = row['zab']
            dla._components= []
            # Correct coordinates now
            sep2 = coord.separation(xq100_coords)
            imin2 = np.argmin(sep2)
            if sep2[imin2] > 0.2*u.arcsec:
                pdb.set_trace()
            dla.coord = xq100_coords[imin2]
            #
            print("Working on DLA {}".format(dla))
        #
        trans = '{:s} {:d}'.format(row['ion'],row['wl'])
        if trans == 'TiII 1910':
            flg_TiII = True
            aline = TiIIa.copy()
        else:
            flg_TiII = False
            aline = AbsLine(trans)
        # Coord matched to the meta table
        aline.attrib['coord'] = dla.coord
        # zem, etc. attrib
        aline.attrib['z'] = row['zab']
        # Scaled N
        if flg_TiII:
            Nscl = 1.
        else:
            Nscl = row['f'] / aline.data['f']
#        pdb.set_trace()
        aline.attrib['logN'] = row['N'] + np.log10(Nscl)
        aline.attrib['sig_logN'] = row['Nerr']
        aline.attrib['flag_N'] = row['flg']
        laa.linear_clm(aline.attrib)
        # Analysis
        aline.analy['vlim'] = [row['vmin'], row['vmax']]*u.km/u.s
        aline.analy['do_analysis'] = row['flg_analy']+1
        aline.analy['spec'] = row['filename']
        # Checks
        np.testing.assert_allclose(aline.wrest.value, row['lambda'], rtol=1e-4)
        try:
            np.testing.assert_allclose(aline.data['f'], row['f'], rtol=1e-4)
        except AssertionError:
            pass
            #print("f-values do not closely match for trans={:s}".format(trans))
            #print("f_lt={:g}, f_Berg={:g}".format(aline.data['f'], row['f']))
        # Add
        abslines.append(aline)
    # Generate components and add them in
    comps = ltiu.build_components_from_abslines(abslines, chk_vel=False)
    for comp in comps:
        comp.synthesize_colm(overwrite=True)
        dla.add_component(comp)
    #pdb.set_trace()

/Users/xavier/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: UserWarning: DLA J0003-2603_z3.390 has a large coordinate offset = [ 3.06824416] arcsec


Working on DLA <DLASystem: 00:03:22.7964 -26:03:19.44, 3.39, NHI=21.4, Z/H=0>


/Users/xavier/local/Python/linetools/linetools/isgm/abscomponent.py:425: UserWarning: Absline <AbsLine: SiII 1304, wrest=1304.3702 Angstrom> has flag=0.  Hopefully you expected that
  warnings.warn("Absline {} has flag=0.  Hopefully you expected that".format(str(aline)))
/Users/xavier/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: UserWarning: DLA J0006-6208_z3.203 has a large coordinate offset = [ 2.90812249] arcsec


Working on DLA <DLASystem: 00:06:51.6079 -62:08:00.78, 3.203, NHI=20.9, Z/H=0>


/Users/xavier/anaconda/lib/python2.7/site-packages/astropy/units/quantity.py:839: RuntimeWarning: divide by zero encountered in true_divide
  return super(Quantity, self).__rtruediv__(other)
/Users/xavier/anaconda/lib/python2.7/site-packages/astropy/units/quantity.py:822: RuntimeWarning: divide by zero encountered in true_divide
  return super(Quantity, self).__truediv__(other)
/Users/xavier/anaconda/lib/python2.7/site-packages/astropy/units/quantity.py:822: RuntimeWarning: invalid value encountered in true_divide
  return super(Quantity, self).__truediv__(other)
/Users/xavier/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: UserWarning: DLA J0006-6208_z3.775 has a large coordinate offset = [ 2.90812249] arcsec


...

/Users/xavier/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: UserWarning: DLA J2239-0552_z4.080 has a large coordinate offset = [ 2.3492306] arcsec


Working on DLA <DLASystem: 22:39:53.6266 -05:52:21.36, 4.0805, NHI=20.6, Z/H=0>


/Users/xavier/local/Python/linetools/linetools/isgm/abscomponent.py:425: UserWarning: Absline <AbsLine: FeII 2600, wrest=2600.1729 Angstrom> has flag=0.  Hopefully you expected that
  warnings.warn("Absline {} has flag=0.  Hopefully you expected that".format(str(aline)))
/Users/xavier/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: UserWarning: DLA J2344+0342_z3.220 has a large coordinate offset = [ 2.82741675] arcsec


Working on DLA <DLASystem: 23:44:03.0545 03:42:24.336, 3.22, NHI=21.3, Z/H=0>


/Users/xavier/local/Python/linetools/linetools/isgm/abscomponent.py:425: UserWarning: Absline <AbsLine: ZnII 2026, wrest=2026.1370 Angstrom> has flag=0.  Hopefully you expected that
  warnings.warn("Absline {} has flag=0.  Hopefully you expected that".format(str(aline)))


### Check

In [15]:
ii=5
xq100_dla._abs_sys[ii].fill_ionN()
#dla.fill_ionN()
#dla._ionN
xq100_dla._abs_sys[ii]._ionN

Z,ion,A,Ej,z,vmin,vmax,flag_N,logN,sig_logN
,,,1 / cm,,km / s,km / s,,,
int64,int64,int64,float64,float64,float64,float64,int64,float64,float64
12,1,0,0.0,2.261,-80.0,190.0,1,12.5196324717,0.02
12,2,0,0.0,2.261,-80.0,190.0,2,14.3094315516,0.03
14,2,0,0.0,2.261,-80.0,190.0,1,15.3815868227,0.02
20,2,0,0.0,2.261,-80.0,190.0,3,11.9400000159,0.0
22,2,0,0.0,2.261,-80.0,190.0,1,12.372404105,0.08
24,2,0,0.0,2.261,-80.0,190.0,1,13.3218304059,0.0287814367476
25,2,0,0.0,2.261,-80.0,190.0,1,12.7376064492,0.0339624101946
26,2,0,0.0,2.261,-100.0,190.0,1,14.9602385578,0.05


### Write

In [16]:
dfile = '/Users/xavier/local/Python/pyigm/pyigm/data/DLA/XQ-100/XQ100_dla_v1.0.tar.gz'
xq100_dla.write_survey(dfile)

Wrote: /Users/xavier/local/Python/pyigm/pyigm/data/DLA/XQ-100/XQ100_dla_v1.0.tar.gz


### Read

In [17]:
import warnings
warnings.filterwarnings('ignore')
reload(pyiu)
xq100_dla = pyiu.load_sys_files(dfile, 'DLA', chk_vel=False, chk_data=False, use_coord=True)

Loading systems from /Users/xavier/local/Python/pyigm/pyigm/data/DLA/XQ-100/XQ100_dla_v1.0.tar.gz
Skipping a likely folder: IGM_JSON


## Kinematics

In [18]:
sys.path.append(os.path.abspath('/Users/xavier/paper/XQ100/DLA_II/Analysis/py'))
import xq100_dla_kin as xqkin
kinfile = '/Users/xavier/paper/XQ100/DLA_II/Analysis/xq100_dla_kin_trans.dat'

### Analyze

In [19]:
reload(xqkin)
xqkin.analyze_kin(xq100_dla, kinfile)

Working on <DLASystem: 00:03:22.7964 -26:03:19.44, 3.39, NHI=21.4, Z/H=0>
Measuring kinematics for <AbsLine: CrII 2062, wrest=2062.2340 Angstrom> in <DLASystem: 00:03:22.7964 -26:03:19.44, 3.39, NHI=21.4, Z/H=0>
Working on <DLASystem: 00:06:51.6079 -62:08:00.78, 3.203, NHI=20.9, Z/H=0>
Measuring kinematics for <AbsLine: FeII 1608, wrest=1608.4511 Angstrom> in <DLASystem: 00:06:51.6079 -62:08:00.78, 3.203, NHI=20.9, Z/H=0>
Working on <DLASystem: 00:06:51.6079 -62:08:00.78, 3.775, NHI=21, Z/H=0>
Measuring kinematics for <AbsLine: FeII 1608, wrest=1608.4511 Angstrom> in <DLASystem: 00:06:51.6079 -62:08:00.78, 3.775, NHI=21, Z/H=0>
Working on <DLASystem: 00:34:54.714 16:39:18.216, 3.755, NHI=20.4, Z/H=0>
Measuring kinematics for <AbsLine: FeII 2586, wrest=2586.6500 Angstrom> in <DLASystem: 00:34:54.714 16:39:18.216, 3.755, NHI=20.4, Z/H=0>
Working on <DLASystem: 00:34:54.714 16:39:18.216, 4.2523, NHI=20.6, Z/H=0>
Measuring kinematics for <AbsLine: SiII 1526, wrest=1526.7070 Angstrom> in <D

### Table

In [20]:
sys.path.append(os.path.abspath('/Users/xavier/paper/XQ100/DLA_II/Tables/py'))
import xq100_dlaII_tables as xqtab

In [21]:
# Load
xqfile = pyigm.__path__[0]+'/data/DLA/XQ-100/XQ100_dla_v1.1.tar.gz'
xq100_dla = pyiu.load_sys_files(xqfile, 'DLA', chk_vel=False, chk_data=False, use_coord=True)

Loading systems from /Users/xavier/local/Python/pyigm/pyigm/data/DLA/XQ-100/XQ100_dla_v1.1.tar.gz
Skipping a likely folder: IGM_JSON


In [22]:
xqtab.mktab_kin(xq100_dla, outfil='/Users/xavier/paper/XQ100/DLA_II/Tables/tab_kin.tex')

All done
